In [ ]:
import keras
from keras.models import load_model, model_from_json
import sys
import matplotlib.pyplot as plt
%matplotlib inline
sys.path.append("./Func")

In [ ]:
from loading_data import DataLoader_base
from building_network import NetworkBuild_base
from run_network import NetworkRun_base
from editing_network import NetworkEdit_base

In [ ]:
import os

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
test_net = NetworkEdit_base()

In [ ]:
load_path = "./models/DnCNN_sigma25/model.json"
test_net.LoadModel("ARCHITECTURE",load_path)
print(test_net.model.summary())

In [ ]:
load_path = "./models/DnCNN_sigma25/model.h5"
test_net.LoadModel("WEIGHTS",load_path)
print(test_net.model.summary())

In [ ]:
print(test_net.model.input_shape,test_net.model.output_shape)

In [ ]:
test_image_path = "./data/Test/Set12/"
image_name = "01.png"

In [ ]:
# test_image_path = "/home/weixin/Documents/MyProjects/CBSD68-dataset/CBSD68/"
# noisy_level = "noisy10"
# test_image_dir = os.path.join(test_image_path,noisy_level)
# ground_truth_dir = os.path.join(test_image_path,"original_png")

In [ ]:
from skimage.measure import compare_psnr, compare_ssim
from skimage.io import imread, imsave
import numpy as np

In [ ]:
x = np.asarray(imread(os.path.join(test_image_path,image_name)),dtype=np.float32)/ 255.0
plt.imshow(x)
print(x.shape)

In [ ]:
np.random.seed(seed=0) # for reproducibility
y = x + np.random.normal(0, 25/255.0, x.shape) # Add Gaussian noise without clipping
y = y.astype(np.float32)
plt.imshow(y)
print(y.shape)

In [ ]:
y = np.reshape(y,(1,256,256,1))
x_predict = test_net.model.predict(y)
print(x_predict.shape)

In [ ]:
x_predict = np.reshape(x_predict,(256,256))
plt.imshow(x_predict)

In [ ]:
psnr_x_ = compare_psnr(x, x_predict)
ssim_x_ = compare_ssim(x, x_predict)
print(psnr_x_,ssim_x_)

In [ ]:
from keras.layers import LocallyConnected2D,Multiply, Input
from keras.models import Sequential

In [ ]:
from keras import backend as K
from keras.engine.topology import Layer

class AdLayer(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(AdLayer, self).__init__(**kwargs)

    def build(self, input_shape):
#         print(type(input_shape),input_shape)
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=self.output_dim,
                                      initializer='uniform',
                                      trainable=True)
        super(AdLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        return x*self.kernel

    def compute_output_shape(self, input_shape):
        return self.output_dim

In [ ]:
del model
print(len(test_net.model.layers))
model = Sequential()
model.add(AdLayer(input_shape=(256,256,1),output_dim=(256,256,1)))
for i in range(4):
    model.add(test_net.model.layers[i])

In [ ]:
print(model.summary())

In [ ]:
print(model.output_shape)

In [ ]:
from keras.layers import  Input,Conv2D,BatchNormalization,Activation,Subtract
from keras.models import Model, load_model
def DnCNN(depth,filters=64,image_channels=1, use_bnorm=True):
    layer_count = 0
    inpt = Input(shape=(256,256,image_channels),name = 'input'+str(layer_count))
    # 1st layer, Conv+relu
    layer_count += 1
    x = Conv2D(filters=filters, kernel_size=(3,3), strides=(1,1),kernel_initializer='Orthogonal', padding='same',name = 'conv'+str(layer_count))(inpt)
    layer_count += 1
    x = Activation('relu',name = 'relu'+str(layer_count))(x)
    # depth-2 layers, Conv+BN+relu
    for i in range(depth-2):
        layer_count += 1
        x = Conv2D(filters=filters, kernel_size=(3,3), strides=(1,1),kernel_initializer='Orthogonal', padding='same',use_bias = False,name = 'conv'+str(layer_count))(x)
        if use_bnorm:
            layer_count += 1
            #x = BatchNormalization(axis=3, momentum=0.1,epsilon=0.0001, name = 'bn'+str(layer_count))(x) 
        x = BatchNormalization(axis=3, momentum=0.0,epsilon=0.0001, name = 'bn'+str(layer_count))(x)
        layer_count += 1
        x = Activation('relu',name = 'relu'+str(layer_count))(x)  
    # last layer, Conv
    layer_count += 1
    x = Conv2D(filters=image_channels, kernel_size=(3,3), strides=(1,1), kernel_initializer='Orthogonal',padding='same',use_bias = False,name = 'conv'+str(layer_count))(x)
    layer_count += 1
    x = Subtract(name = 'subtract' + str(layer_count))([inpt, x])   # input - noise
    model = Model(inputs=inpt, outputs=x)
    
    return model


In [ ]:
model = DnCNN(17)

In [ ]:
print(model.summary())

In [ ]:
print(len(model.layers))

In [ ]:
test_net = NetworkEdit_base(model=model)

In [ ]:
print(test_net.model.summary())

In [ ]:
load_path = "./models/DnCNN_sigma25/model.h5"
test_net.LoadModel("WEIGHTS",load_path)
print(test_net.model.summary())

In [ ]:
def MyDnCNN(depth,filters=64,image_channels=1, use_bnorm=True):
    layer_count = 0
    inpt = AdLayer(input_shape=(256,256,image_channels),output_dim=(256,256,image_channels))
    # 1st layer, Conv+relu
    layer_count += 1
    x = Conv2D(filters=filters, kernel_size=(3,3), strides=(1,1),kernel_initializer='Orthogonal', padding='same',name = 'conv'+str(layer_count))(inpt)
    layer_count += 1
    x = Activation('relu',name = 'relu'+str(layer_count))(x)
    # depth-2 layers, Conv+BN+relu
    for i in range(depth-2):
        layer_count += 1
        x = Conv2D(filters=filters, kernel_size=(3,3), strides=(1,1),kernel_initializer='Orthogonal', padding='same',use_bias = False,name = 'conv'+str(layer_count))(x)
        if use_bnorm:
            layer_count += 1
            #x = BatchNormalization(axis=3, momentum=0.1,epsilon=0.0001, name = 'bn'+str(layer_count))(x) 
        x = BatchNormalization(axis=3, momentum=0.0,epsilon=0.0001, name = 'bn'+str(layer_count))(x)
        layer_count += 1
        x = Activation('relu',name = 'relu'+str(layer_count))(x)  
    # last layer, Conv
    layer_count += 1
    x = Conv2D(filters=image_channels, kernel_size=(3,3), strides=(1,1), kernel_initializer='Orthogonal',padding='same',use_bias = False,name = 'conv'+str(layer_count))(x)
    layer_count += 1
    x = Subtract(name = 'subtract' + str(layer_count))([inpt, x])   # input - noise
    model = Model(inputs=inpt, outputs=x)
    
    return model


In [ ]:
new_model = MyDnCNN(17)

In [ ]:
from keras.engine.topology import InputSpec
from keras import backend as K
from keras.engine.topology import Layer
class MyLayer(Layer):

    def __init__(self, **kwargs):
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.input_spec = [InputSpec(shape=input_shape)]
        self.kernel = self.add_weight(name='kernel', 
                                      shape=self.output_dim,
                                      initializer='glorot_uniform',
                                      trainable=True)
        super(MyLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs):
        return inputs*self.kernel


In [3]:
from keras.engine.topology import InputSpec
a = [InputSpec(shape=(None,256,256,1))]

In [4]:
print(a)

[InputSpec(shape=(None, 256, 256, 1), ndim=4)]
